In [ ]:
#%pip install pandas
#%pip install numpy
#%pip install matplotlib
#%pip install seaborn
#%pip install hypertools
#%pip install scikit-learn
#%pip install tensorflow
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import hypertools as hyp 
from sklearn.linear_model import LogisticRegression # to apply the Logistic regression
from sklearn.model_selection import train_test_split # to split the data into two parts
from sklearn.model_selection import GridSearchCV# for tuning parameter
from sklearn.ensemble import RandomForestClassifier # for random forest classifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
import matplotlib
from matplotlib import pyplot as plt
from sklearn import svm # for Support Vector Machine
from sklearn import metrics # for the check the error and accuracy of the model
import seaborn as sns

%matplotlib inline

pd.set_option('display.max_columns', 100)

In [ ]:
df= pd.read_csv('data\Breast Cancer Wisconsin.csv',  index_col= None, na_values='?')
df.head(6)

# Data Preprocessing

In [ ]:
df.shape

In [ ]:
df.isnull().sum()

In [ ]:
#df.drop('Unnamed: 32', axis=1 , inplace=True)
df.shape

In [ ]:
df.drop('id', axis=1 , inplace=True)
df.shape

In [ ]:
df["diagnosis"].value_counts()


# Feature Selection

In [ ]:
def categorical_to_numeric_diagnosis(x):
    if x=='M':
        return 1
    if x=='B':
        return 0

df['diagnosis']= df['diagnosis'].apply(categorical_to_numeric_diagnosis)
df["diagnosis"].value_counts()

In [ ]:
#Observe correlation to the diagnosis
f, ax = plt.subplots(figsize=(18, 12))
tst = df.corr()['diagnosis'].copy()
tst = tst.drop('diagnosis')
tst.sort_values(inplace=True)
tst.plot(kind="bar", alpha=1)

In [ ]:
#The features looks like they are broken into three main categories, value means, standard deivations, and 'worst'. We can check to see if there are any correlations between these subsets of features
features_mean = df.columns[1:9]
features_se = df.columns[9:19]
features_worst = df.columns[19:]

In [ ]:
#Correlation between sets of features
corr = df[features_se].corr()
g = sns.heatmap(corr, cbar = True, annot=True, annot_kws={'size': 15}, fmt= '.2f', square = True, cmap = 'coolwarm' )
g.set_xticklabels(rotation=90, labels = features_mean, size = 15)
g.set_yticklabels(rotation=0, labels = features_mean, size = 15)
g.set_xticks(np.arange(.5,10.5,1))
plt.rcParams["figure.figsize"] = (17,17)

In [ ]:
corr = df[features_se].corr()
g = sns.heatmap(corr, cbar = True, annot=True, annot_kws={'size': 15}, fmt= '.2f', square = True, cmap = 'coolwarm' )
g.set_xticklabels(rotation=90, labels = features_se, size = 15)
g.set_yticklabels(rotation=0, labels = features_se, size = 15)
g.set_xticks(np.arange(.5,10.5,1))
plt.rcParams["figure.figsize"] = (17,17)

In [ ]:
corr = df[features_worst].corr()
g = sns.heatmap(corr, cbar = True, annot=True, annot_kws={'size': 15}, fmt= '.2f', square = True, cmap = 'coolwarm' )
g.set_xticklabels(      rotation=90, labels = features_worst, size = 15)
g.set_yticklabels(rotation=0, labels = features_worst, size = 15)
g.set_xticks(np.arange(.5,10.5,1))
plt.rcParams["figure.figsize"] = (17,17)

In [ ]:
#Correlation Table
corr = df.corr()
corr = (corr)

corr

In [ ]:
#Correlation Matrix

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(20, 20))
plt.title('Pearson Correlation of Mushroom Features')
# Draw the heatmap using seaborn
sns.heatmap(df.astype(float).corr(),linewidths=0.7,vmax=1.0, square=True, annot=True)